<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/manage_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.4 MB/s eta 0:00:00


In [2]:
from web3 import Web3
from eth_account import Account
import requests
import json
import time
import decimal
import numpy as np
import math

In [23]:
from web3 import exceptions
from eth_abi import encode

In [4]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/demo"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
my_wallet_address = account.address

print("Connected:", w3.is_connected())
print("Your address:", my_wallet_address)

Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [5]:
def fetch_abi(contract_address: str, api_key: str,inp_chain_id:int):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "module": "contract",
        "action": "getabi",
        "address": contract_address,
        "apikey": api_key,
        "chainid":inp_chain_id
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if data.get("status") != "1":
        raise Exception(f"Error getting ABI: {data.get('message')}")
    abi = json.loads(data["result"])
    return abi

In [6]:
etherscan_apikey = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
CHAIN_ID = 11155111  # Sepolia

In [7]:
position_manager_address = "0x1238536071E1c677A632429e3655c799b22cDA52"
position_manager_address = w3.to_checksum_address(position_manager_address)
position_manager_abi = fetch_abi(position_manager_address,etherscan_apikey,CHAIN_ID)
position_manager_contract = w3.eth.contract(address=position_manager_address, abi=position_manager_abi)

In [19]:
def check_liquidity_v3_nft(inp_wallet_address,inp_postion_manger_contract):
  pm_contract=inp_postion_manger_contract
  address=inp_wallet_address
  nfts=[]
  try:
    balance = pm_contract.functions.balanceOf(address).call()
    print(f"تعداد NFT پوزیشن‌ها: {balance}")


    if balance > 0:
        for i in range(balance):

            token_id = pm_contract.functions.tokenOfOwnerByIndex(address, i).call()

            pos = pm_contract.functions.positions(token_id).call()
            dct={'token_id':token_id,'token0':pos[2],'token1':pos[3],'fee_tier':pos[4],'liquidity':pos[7],'tick_range':[pos[5],pos[6]],'fee0':pos[10],'fee1':pos[11]}
            nfts.append(dct)
        return nfts
    else:
        print("lp nft not found")

  except Exception as e:
    print(f"error in read data --> {e}")

In [20]:
V3_LPS=check_liquidity_v3_nft(my_wallet_address,position_manager_contract)
V3_LPS[0]

تعداد NFT پوزیشن‌ها: 1


{'token_id': 221424,
 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee_tier': 500,
 'liquidity': 33516997100150,
 'tick_range': [186440, 186640],
 'fee0': 0,
 'fee1': 0}

In [18]:
v3factoryaddress="0x0227628f3F023bb0B980b67D528571c95c6DaC1c"
factory_address = w3.to_checksum_address(v3factoryaddress)
factory_abi =fetch_abi(factory_address,etherscan_apikey,CHAIN_ID)
factory=w3.eth.contract(address=factory_address, abi=factory_abi)

In [22]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [21]:
def pool_current_info(inp_pool_contract):
  token0_address = inp_pool_contract.functions.token0().call()
  token1_address = inp_pool_contract.functions.token1().call()
  slot0 = inp_pool_contract.functions.slot0().call()
  liquidity = inp_pool_contract.functions.liquidity().call()
  fee = inp_pool_contract.functions.fee().call()
  tickspacing = inp_pool_contract.functions.tickSpacing().call()
  return {'token0_address':token0_address,'token1_address':token1_address,'sqrtprice':slot0[0],'current_tick':slot0[1],'liquidity':liquidity,'fee_tier':fee,'tick_spacing':tickspacing}

In [29]:
def make_swap(input_token_address,output_token_address,amount_input:float,inp_wallet,inp_pool_address,swap_router_address,mode:str,inp_private_key):
  input_token_address=w3.to_checksum_address(input_token_address)
  erc20_abi=fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  input_token_contract=w3.eth.contract(address=input_token_address, abi=erc20_abi)
  input_token_contract=w3.eth.contract(address=input_token_address, abi=erc20_abi)
  output_token_address=w3.to_checksum_address(output_token_address)
  pool_address=w3.to_checksum_address(inp_pool_address)
  pool_abi=fetch_abi(pool_address,etherscan_apikey,CHAIN_ID)
  pool_contract=w3.eth.contract(address=pool_address, abi=pool_abi)
  swap_router_address=w3.to_checksum_address(swap_router_address)
  swap_router_abi=fetch_abi(swap_router_address,etherscan_apikey,CHAIN_ID)
  swap_router_contract=w3.eth.contract(address=swap_router_address, abi=swap_router_abi)
  pool_info=pool_current_info(pool_contract)
  amount_in_wei=amount_input*(10**input_token_contract.functions.decimals().call())
  if input_token_address in pool_info.values and output_token_address in pool_info.values:
    params = {
        "tokenIn": input_token_address,
        "tokenOut": output_token_address,
        "fee": pool_info['fee_tier'],
        "recipient": inp_wallet,
        "amountIn": amount_in_wei,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0}
    nonce = w3.eth.get_transaction_count(inp_wallet)

    swap_tx = swap_router_contract.functions.exactInputSingle(params).build_transaction({
      "from": inp_wallet,
      "gasPrice": w3.eth.gas_price})

    est_gas_need=w3.eth.estimate_gas(swap_tx)

    if mode=='send_raw':

      swap_tx = swap_router_contract.functions.exactInputSingle(params).build_transaction({
        "from": inp_wallet,
        "nonce": nonce,
        "gas": est_gas_need,
        "gasPrice": w3.eth.gas_price})
      signed_tx = w3.eth.account.sign_transaction(swap_tx, inp_private_key)
      tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
      return w3.to_hex(tx_hash)

    elif mode=='simulate':
      return est_gas_need
    else:
      return False


In [25]:
def check_token_balance(token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  eth_balance = w3.eth.get_balance(inp_address)
  balances.append(["ETH", w3.from_wei(eth_balance, 'ether')])
  return balances

In [32]:
def calculate_price_by_tick(tick:int,decimal_0:int,decimal_1:int):
  return (10**(decimal_0-decimal_1))*(1.0001**tick)

In [33]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [37]:
def init_amounts_liquid(inp_sqrt_price,
                       inp_decimal0, inp_decimal1, inp_token0_real_price: float,
                       inp_token1_real_price: float,inp_price_low,inp_price_high ,inp_hypo_capital: float):

    current_price = calculate_price_from_sqrt_price(inp_sqrt_price, decimal0, decimal1)
    #current_price = (price_low+price_high)/2
    s = np.sqrt(current_price)
    su = np.sqrt(inp_price_high)
    sl = np.sqrt(inp_price_low)

    amount_0 = (inp_hypo_capital / 2.0) / inp_token0_real_price
    amount_1 = (inp_hypo_capital / 2.0) / inp_token1_real_price

    if su == s:
        l_0 = np.inf
    else:
        l_0 = amount_0 * s * su / (su - s)

    if s == sl:
        l_1 = np.inf
    else:
        l_1 = amount_1 / (s - sl)

    l = min(l_0, l_1)

    final_amount_0 = l * (su - s) / (s * su)
    final_amount_1 = l * (s - sl)

    return l, final_amount_0, final_amount_1

In [31]:
def calculate_new_amount_token_mix(current_price: float, lower: float, upper: float, liquidity: float):
    s = np.sqrt(current_price)
    su = np.sqrt(upper)
    sl = np.sqrt(lower)

    amount_0 = 0.0
    amount_1 = 0.0

    if current_price < lower:
        amount_0 = liquidity * ((1.0 / sl) - (1.0 / su))
        amount_1 = 0.0
    elif current_price > upper:
        amount_0 = 0.0
        amount_1 = liquidity * (su - sl)
    else:
        amount_0 = liquidity * ((1.0 / s) - (1.0 / su))
        amount_1 = liquidity * (s - sl)

    return amount_0, amount_1

In [ ]:
router_address=w3.to_checksum_address('0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E')
collect_estimated_gas=0
burn_estimated_gas=0
decrease_estimated_gas=0
mint_estimated_gas=0
swap_estimated_gas=0

for lp_nft in V3_LPS:
  token0_address=lp_nft['token0']
  token1_address=lp_nft['token1']
  fee_tier=lp_nft['fee_tier']
  erc20_abi=fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  token0_contract=w3.eth.contract(address=token0_address, abi=erc20_abi)
  token1_contract=w3.eth.contract(address=token1_address, abi=erc20_abi)
  decimal0=token0_contract.functions.decimals().call()
  decimal1=token1_contract.functions.decimals().call()
  pool_address_v=factory.functions.getPool(token0_address, token1_address, fee_tier).call()
  pool_abi=fetch_abi(pool_address_v,etherscan_apikey,CHAIN_ID)
  pool_contract=w3.eth.contract(address=pool_address_v, abi=pool_abi)
  pool_info=pool_current_info(pool_contract)
  fee_balance_in_eth=lp_nft['fee0']*calculate_price_from_sqrt_price(pool_info['sqrtprice'],decimal0,decimal1)+lp_nft['fee1']
  POSITION_TOKEN_ID = lp_nft['token_id']
  liquidity_amount = lp_nft['liquidity']

  # A. Decrease Liquidity
  params_decrease = {
            "tokenId": POSITION_TOKEN_ID,
            "liquidity": liquidity_amount,
            "amount0Min": 0,
            "amount1Min": 0,
            "deadline": int(time.time()) + 1200}

  decrease_tx = position_manager_contract.functions.decreaseLiquidity(params_decrease).build_transaction({
            "from": my_wallet_address,
            "gasPrice": w3.eth.gas_price})

  decrease_estimated_gas = w3.eth.estimate_gas(decrease_tx)

  # B. Collect Fees/Tokens
  MAX_UINT128 = 2**128 - 1
  params_collect = {
            "tokenId": POSITION_TOKEN_ID,
            "recipient": my_wallet_address,
            "amount0Max": MAX_UINT128,
            "amount1Max": MAX_UINT128}

  collect_tx = position_manager_contract.functions.collect(params_collect).build_transaction({
            "from": my_wallet_address,
            "gasPrice": w3.eth.gas_price})

  collect_estimated_gas = w3.eth.estimate_gas(decrease_tx)

  # C. Burn NFT
  burn_tx = position_manager_contract.functions.burn(POSITION_TOKEN_ID).build_transaction({
            "from": my_wallet_address,
            "gasPrice": w3.eth.gas_price})

  burn_estimated_gas = w3.eth.estimate_gas(burn_tx)
  ctb=check_token_balance([token0_address,token1_address])
  swap_estimated_gas=make_swap(token0_address,token1_address,ctb[0][1]/10,my_wallet_address,pool_address_v,router_address,'simulate',PRIVATE_KEY)

  def get_nearest_valid_tick(tick, spacing):
    return (tick // spacing) * spacing

  valid_current_tick = get_nearest_valid_tick(pool_info['current_tick'], pool_info['tick_spacing'])



  tick_lower = valid_current_tick - (10 * pool_info['tick_spacing'])
  tick_upper = valid_current_tick + (10 * pool_info['tick_spacing'])

  # ----------------------------------------------
  req_amount0=ctb[0][1]
  req_amount1=ctb[1][1]
  params = {
    "token0": token0_address,
    "token1": token1_address,
    "fee": pool_info['fee_tier'],
    "tickLower": tick_lower,
    "tickUpper": tick_upper,
    "amount0Desired": int(req_amount0 * 10**decimal0),
    "amount1Desired": w3.to_wei(req_amount1, "ether"),
    "amount0Min": 0,
    "amount1Min": 0,
    "recipient": my_wallet_address,
    "deadline": int(time.time()) + 1200}

  # ساخت تراکنش
  tx = position_manager_contract.functions.mint(params).build_transaction({
    "from": my_wallet_address,
    "gasPrice": w3.eth.gas_price})

  mint_estimated_gas = w3.eth.estimate_gas(tx)
  all_gas_need=decrease_estimated_gas+collect_estimated_gas+burn_estimated_gas+mint_estimated_gas+swap_estimated_gas
  if pool_info['current_tick']>lp_nft['tick_range'][0] and pool_info['current_tick']<lp_nft['tick_range'][1] or w3.to_wei(fee_balance_in_eth, 'ether')>(all_gas_need*w3.eth.gas_price):
    pass
  else:
    current_nonce = w3.eth.get_transaction_count(my_wallet_address)

    # A. Decrease Liquidity
    params_decrease = {
            "tokenId": POSITION_TOKEN_ID,
            "liquidity": liquidity_amount,
            "amount0Min": 0,
            "amount1Min": 0,
            "deadline": int(time.time()) + 1200}

    decrease_tx = position_manager_contract.functions.decreaseLiquidity(params_decrease).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce, # استفاده از نانس محلی
            "gas": decrease_estimated_gas,
            "gasPrice": w3.eth.gas_price})

    signed_decrease_tx = w3.eth.account.sign_transaction(decrease_tx, PRIVATE_KEY)
    hash_decrease = w3.eth.send_raw_transaction(signed_decrease_tx.raw_transaction)
    print(f"Decrease TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_decrease)}")

    time.sleep(10)
    current_nonce = w3.eth.get_transaction_count(my_wallet_address)
    # B. Collect Fees/Tokens
    MAX_UINT128 = 2**128 - 1
    params_collect = {
            "tokenId": POSITION_TOKEN_ID,
            "recipient": my_wallet_address,
            "amount0Max": MAX_UINT128,
            "amount1Max": MAX_UINT128}

    collect_tx = position_manager_contract.functions.collect(params_collect).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce, # استفاده از نانس محلی
            "gas": collect_estimated_gas,
            "gasPrice": w3.eth.gas_price})

    signed_collect_tx = w3.eth.account.sign_transaction(collect_tx, PRIVATE_KEY)
    hash_collect = w3.eth.send_raw_transaction(signed_collect_tx.raw_transaction)
    print(f"Collect TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_collect)}")

    time.sleep(10)
    # C. Burn NFT
    burn_tx_active = position_manager_contract.functions.burn(POSITION_TOKEN_ID).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce, # استفاده از نانس محلی
            "gas": burn_estimated_gas,
            "gasPrice": w3.eth.gas_price})
    signed_burn_tx_active = w3.eth.account.sign_transaction(burn_tx_active, PRIVATE_KEY)
    hash_burn_active = w3.eth.send_raw_transaction(signed_burn_tx_active.raw_transaction)
    print(f"Burn TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_burn_active)}")

    ctb=check_token_balance([token0_address,token1_address])
    swap_estimated_gas=make_swap(token0_address,token1_address,ctb[0][1]/10,my_wallet_address,pool_address_v,router_address,'simulate',PRIVATE_KEY)

    def get_nearest_valid_tick(tick, spacing):
      return (tick // spacing) * spacing

    valid_current_tick = get_nearest_valid_tick(pool_info['current_tick'], pool_info['tick_spacing'])

    a=valid_current_tick - (1 * pool_info['tick_spacing'])
    b=tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])
    if valid_current_tick>(2*((a+b)/3)):
      tick_lower = valid_current_tick - (1 * pool_info['tick_spacing'])
      tick_upper = valid_current_tick + (2 * pool_info['tick_spacing'])
    elif valid_current_tick<(1)*((a+b)/3):
      tick_lower = valid_current_tick - (2 * pool_info['tick_spacing'])
      tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])
    else:
      tick_lower = valid_current_tick - (1 * pool_info['tick_spacing'])
      tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])

    # ----------------------------------------------
    req_amount0=ctb[0][1]
    req_amount1=ctb[1][1]
    price_lower=calculate_price_by_tick(tick_lower,decimal0,decimal1)
    price_upper=calculate_price_by_tick(tick_upper,decimal0,decimal1)
    eth_sepolia_price=1/calculate_price_from_sqrt_price(pool_info['sqrtprice'],decimal0,decimal1)
    capital=req_amount0*1+req_amount1*eth_sepolia_price
    L=init_amounts_liquid(pool_info['sqrtprice'],decimal0,decimal1,1,eth_sepolia_price,price_lower,price_upper,)
    params = {
      "token0": token0_address,
      "token1": token1_address,
      "fee": pool_info['fee_tier'],
      "tickLower": tick_lower,
      "tickUpper": tick_upper,
      "amount0Desired": int(L[1] * 10**decimal0),
      "amount1Desired": w3.to_wei(L[2], "ether"),
      "amount0Min": 0,
      "amount1Min": 0,
      "recipient": my_wallet_address,
      "deadline": int(time.time()) + 1200}

    # ساخت تراکنش
    time.sleep(10)
    tx = position_manager_contract.functions.mint(params).build_transaction({
    "from": my_wallet_address,
    "nonce": w3.eth.get_transaction_count(my_wallet_address),
    "gas": mint_estimated_gas,
    "gasPrice": w3.eth.gas_price})

    signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

    print("TX SENT:", w3.to_hex(tx_hash))
